In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

In [2]:
# setting the path for Data from data/housing folder
DATA_FILE_TRAIN = './data/Boston.csv'
#setting the random seed 
np.random.seed(42)
# Loading the dataset
train_data = pd.read_csv(DATA_FILE_TRAIN)
print(train_data.shape)
print(train_data.head())


(506, 15)
   ID     crim    zn  indus  chas    nox     rm   age     dis  rad  tax  \
0   1  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
1   2  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
2   3  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
3   4  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
4   5  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   

   ptratio   black  lstat  medv  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2  


In [3]:
BATCH_SIZE_1 = 101
BATCH_SIZE_2 = 61

In [4]:
# To decide on the bin values
print(train_data['medv'].max())
print(train_data['medv'].min())

50.0
5.0


In [5]:

bins = [0,30,50]
labels = [0,1]
train_data['medv'] = pd.cut(train_data['medv'], bins=bins, labels=labels)
print(train_data.head())
print(len(train_data.columns))

   ID     crim    zn  indus  chas    nox     rm   age     dis  rad  tax  \
0   1  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
1   2  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
2   3  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
3   4  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
4   5  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   

   ptratio   black  lstat medv  
0     15.3  396.90   4.98    0  
1     17.8  396.90   9.14    0  
2     17.8  392.83   4.03    1  
3     18.7  394.63   2.94    1  
4     18.7  396.90   5.33    1  
15


In [6]:
print(f"The target class ratio is {Counter(train_data['medv']) }")

NameError: name 'Counter' is not defined

In [ ]:
id_col = ['ID']
categorical_features = ['chas'] 
target_feature = 'medv'

dropped_cols = id_col+categorical_features
train_data = train_data.drop(dropped_cols, axis=1)
all_features = train_data.columns.tolist()  #this will not have 'chas' and 'ID'

numerical_features = list(set(all_features)- set([target_feature]))
#print(len(numerical_features))


In [ ]:
train_data_inp = train_data[numerical_features]
train_data_tar = train_data[target_feature]
Trn_input,  Val_inp, Trn_target,Val_target = train_test_split(train_data_inp, train_data_tar, test_size=0.2,random_state=123)
# Train_data has our training dataset and Valid_data has our validation dataset.
Train_data = pd.concat([Trn_input, pd.DataFrame(Trn_target)], axis=1)
Valid_data = pd.concat([Val_inp, pd.DataFrame(Val_target)], axis=1)
print(Train_data.shape)
print(Valid_data.shape)

In [ ]:
class oversampdata(Dataset):

    def __init__(self, data):
        self.inp_data = torch.FloatTensor(data.values.astype('float')[:, :-1])
        self.out_data = data.values[:, -1]
        self.out_data = torch.FloatTensor(get_categorical(self.out_data).astype('float'))
        print(self.out_data.shape)
    def __len__(self):
        return len(self.inp_data)

    def __getitem__(self, index):
        #target = self.out_data[ind]
        #data_val = self.data[index] [:-1]
        return self.inp_data[index],self.out_data[index]

In [ ]:
def get_categorical(arr):
    out_arr = np.zeros(((len(arr), 2)))
    for i,val in enumerate(arr):
        out_arr[i][int(val)] = 1
    #print(np.argmax(out_arr, axis =1))
    return out_arr

In [ ]:
# training and validation dataset 
train_dataset = oversampdata(Train_data)
valid_dataset = oversampdata(Valid_data)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_1, shuffle=True, **kwargs)
test_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE_2, shuffle=True, **kwargs)
